In [1]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke
import re

In [2]:
# Сюда вводим искомую должность
# inp_vacancy = 'Java junior'
inp_vacancy = 'инженер'

# А сюда колличество страниц для сбора данных с сайта 
num_of_pages = 7

In [3]:
vacancy_list = []
page = 1

In [4]:
url = 'https://www.hh.ru'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'}
page_button = f'/search/vacancy?fromSearchLine=true&text={inp_vacancy}&from=suggest_post&search_field=name&page={page}'

In [5]:
%%time
for _ in range(num_of_pages):
    try:
        response = requests.get(url+page_button, headers=headers)
        dom = bs(response.text, 'html.parser')
        vacancies = dom.find_all('div', {'class': 'vacancy-serp-item'})
        for vacancy in vacancies:

            base = {}
            base['site'] = url
            base['title'] = vacancy.find('a', {'class': 'bloko-link', 'data-qa':"vacancy-serp__vacancy-title"}).text
            base['link'] = vacancy.find('a', {'class': 'bloko-link', 'data-qa':"vacancy-serp__vacancy-title"})['href']
            base['_id'] = int(re.findall(r'\d+', base['link'])[0])
            
            try:
                raw_salary = (vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text)
                li = re.findall(r"\d+", raw_salary)

                if raw_salary[0] == 'о':
                    base['min_salary'] = int(li[0]+li[1])
                    base['max_salary'] = None
                    base['value'] = re.findall(r'\w+\.?$', raw_salary)[0]

                elif raw_salary[0] == 'д':
                    base['min_salary'] = None
                    base['max_salary'] = int(li[0]+li[1])
                    base['value'] = re.findall(r'\w+\.?$', raw_salary)[0]

                else:
                    base['min_salary'] = int(li[0]+li[1])
                    base['max_salary'] = int(li[2]+li[3])
                    base['value'] = re.findall(r'\w+\.?$', raw_salary)[0]

            except:
                base['min_salary'] = None
                base['max_salary'] = None
                base['value'] = None


            vacancy_list.append(base)
        
        page += 1
        
    except:
        break

Wall time: 10.3 s


In [6]:
len(vacancy_list)

359

In [7]:
client = MongoClient('127.0.0.1', 27017)
db = client['vacancy_base']
base_1 = db.base_1

In [8]:
dke_counter = 0
for idx, _ in enumerate(vacancy_list):
    
    try:
        base_1.insert_one(vacancy_list[idx])
    except dke:
        dke_counter += 1
print(f'Повторений ваканский в vacancy_list найдено: {dke_counter}')

Повторений ваканский в vacancy_list найдено: 300


In [9]:
base_1.count_documents({})

59

In [10]:
def find_salary(salary):
    count = 0
    for item in base_1.find({'$or':
                             [{'min_salary': {'$gte': salary}}, 
                              {'max_salary': {'$gte': salary}}]}):
        pprint(item)
        count += 1
    print(f'\nВсего {count} вакансий c зароботной платы свыше {salary}.')
        

In [11]:
find_salary(150000)

{'_id': 49253637,
 'link': 'https://ufa.hh.ru/vacancy/49253637?query=%D0%B8%D0%BD%D0%B6%D0%B5%D0%BD%D0%B5%D1%80',
 'max_salary': None,
 'min_salary': 150000,
 'site': 'https://www.hh.ru',
 'title': 'Ведущий инженер ПТО',
 'value': 'руб.'}
{'_id': 47352945,
 'link': 'https://ufa.hh.ru/vacancy/47352945?query=%D0%B8%D0%BD%D0%B6%D0%B5%D0%BD%D0%B5%D1%80',
 'max_salary': 150000,
 'min_salary': 80000,
 'site': 'https://www.hh.ru',
 'title': 'Ведущий инженер по ППР и тех картам',
 'value': 'руб.'}
{'_id': 46738124,
 'link': 'https://ufa.hh.ru/vacancy/46738124?query=%D0%B8%D0%BD%D0%B6%D0%B5%D0%BD%D0%B5%D1%80',
 'max_salary': 150000,
 'min_salary': 65000,
 'site': 'https://www.hh.ru',
 'title': 'Техник ПТО \\ Инженер ПТО (Ведущий инженер ПТО)',
 'value': 'руб.'}
{'_id': 49226396,
 'link': 'https://ufa.hh.ru/vacancy/49226396?query=%D0%B8%D0%BD%D0%B6%D0%B5%D0%BD%D0%B5%D1%80',
 'max_salary': 150000,
 'min_salary': 120000,
 'site': 'https://www.hh.ru',
 'title': 'Ведущий инженер ПТО',
 'value': 'руб

In [12]:
db.drop_collection('base_1')
base_1.count_documents({})

0